In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas
!pip install contextily
import pandas as pd
import geopandas as gpd
import fiona
import contextily as cx
import rtree
from shapely.geometry import Point, Polygon
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import random
import multiprocessing as mp
import dill
import progressbar
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load data

In [109]:
# Change dir accordingly
root_path = '/content/drive/MyDrive/BoxMigration/Geodata_Backup/'
welldata_path = root_path + 'well2field_2023/csv/'
result_path = root_path + 'well2field_2023/field_output/'

In [107]:
# csv to geodf
def cvt_point_to_geodf(df, lonname, latname):
    df['coords'] = list(zip(df[lonname],df[latname]))
    df['coords'] = df['coords'].apply(Point)
    gdf = gpd.GeoDataFrame(df, geometry='coords')
    
    return gdf

# load well
def load_Well(welldata_path, country):
    well = pd.read_csv(welldata_path + country + "_well.csv")
    well = well[['id_well_pk', 'country_name', 'well_associated_field', 'well_th_long_decimal_wgs84', 'well_th_lat_decimal_wgs84']] # optional
    well = cvt_point_to_geodf(well, 'well_th_long_decimal_wgs84', 'well_th_lat_decimal_wgs84')
    well = well.set_crs(epsg=4326)
    
    return well

# Generate well buffer for wells
def buffer_Well(well, dist):
    # well_buffer = well.to_crs("EPSG:32634")
    well_buffer = well.to_crs("EPSG:3857")
    well_buffer.geometry = well_buffer.geometry.buffer(dist)
    well_buffer = well_buffer.to_crs("EPSG:4326")
    
    return well_buffer

# Dissolve well buffers with the same field name into one field
def dissolve_buffer(well_buffer):
    field = well_buffer.dissolve(by = 'well_associated_field', aggfunc='first')
    
    return field

## Field generation

In [108]:
# Test
country = "Algeria"
dist = 1000

In [110]:
well = load_Well(welldata_path, country)
well_buffer = buffer_Well(well, dist)
field = dissolve_buffer(well_buffer)

field.to_file(result_path + country + ".shp", crs = "EPSG:4326")
field

TypeError: ignored